<h2>Import Dependencies</h2>

In [2]:
import urllib
from bs4 import BeautifulSoup as bs
import nltk
import pandas  as pd
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from nltk import ngrams
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
from nltk.cluster.util import cosine_distance
from gensim.models import Word2Vec
import numpy as np
from nltk.tokenize import RegexpTokenizer
from operator import itemgetter 

<h2> Parse and clean data </h2>

In [3]:
def clean_data(html,baseurl):
    soup = bs(html, 'html.parser')
    if baseurl == 'i':
        entries = soup.find_all('div',{'class':'description'})
    elif baseurl == 't':
        entries = soup.find_all('div',{'itemprop':'articleBody'})
    elif baseurl == 'h':
        entries = soup.find_all('div',{'itemprop':'articlebody'})
    for each in entries:
            if each.figure:
                each.figure.decompose()
    content = []
    for e in entries:
         content.extend(e.find_all("p"))
    
    text = ""
    for each in content:
        text = text + each.get_text() +" "

    text = text.encode('utf-8').decode("unicode_escape").encode('ascii','ignore')
    text = nltk.sent_tokenize(text)    
    return text

<h2> Luhn's Approach </h2>
<h3> Extraction </h3>

In [4]:
def word_scores(text):
    # tokenize the article
    tokenizer = RegexpTokenizer(r'\w+')
    word_list=[]
    for t in text:
        word_list.extend(tokenizer.tokenize(t))


    # remove stop words from the article
    filtered_words = [word for word in word_list if word.lower() not in stopwords.words('english')]
    total_words = len(filtered_words)
    words = filtered_words

    # find n-gram probability for filtered words
    filtered_words  = []
    for each in Counter(words).items():
        filtered_words.append([each[0] , float(each[1])/float(total_words)])

    # identify important words
    important_words = []
    for item in filtered_words:
        if item[1]>0.003:
            important_words.append(item[0])

    while '. ' in text:
        text = text.replace('. ','\n') 

    while "\n" in text:
        text = text.split("\n")
        
    vect = TfidfVectorizer(stop_words='english')
    dtm = vect.fit_transform(text)
    features = vect.get_feature_names()
    
    scores = zip(vect.get_feature_names(),
                 np.asarray(dtm.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    sorted_scores = dict(sorted_scores)
    return sorted_scores

<h3> Sentence Score </h3>

In [5]:
def sentence_score(text,sorted_scores):
    sentence_score = []
    for each in text:
        score = 0.0
    #     each = each.
        for word in each.split(" "):
            #print word
            word = word.lower()
            if word in sorted_scores:
    #             print "Hello"
                score += sorted_scores[word]
    #             print word_scores[word]
        sentence_score.append([each,score])    
    return sentence_score

<h2> TextRank </h2>
<h3> Sentence Sim </h3>

In [6]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
#     print vector1,vector2
    return 1 - cosine_distance(vector1, vector2)

<h3> Similarity with Word2Vec </h3>

In [7]:
def sim_scores(text):
    v1 = np.zeros(150)
    scores = []
    model = Word2Vec.load('model')
    for s in text:
        tokenizer = RegexpTokenizer(r'\w+')
        sentence = tokenizer.tokenize(s)
        for w in sentence:
            if w in model:
                v1 = v1 + model[w]
        scores.append([s,v1])
    
    from nltk.cluster.util import cosine_distance

    sim = np.zeros([len(scores),len(scores)])
    for i in xrange(len(scores)):
        for j in xrange(len(scores)):   
            sim[i][j] = cosine_distance(scores[i][1],scores[j][1])
    return sim

<h3> Build Similarity Matrix </h3>

In [8]:
def build_similarity_matrix(sentences, stopwords=None):
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stopwords)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
 
    return S

<h3> PageRank </h3>

In [9]:
def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs((new_P - P).sum())
        if delta <= eps:
            return new_P
        P = new_P

<h3> TextRank </h3>

In [10]:
def textrank(sentences,method, top_n=4, stopwords=None):
    """
    sentences = a list of sentences [[w11, w12, ...], [w21, w22, ...], ...]
    top_n = how may sentences the summary should contain
    stopwords = a list of stopwords
    """
    if method == "word_count":
        S = build_similarity_matrix(sentences, stopwords)
    elif method == "word2vec":    
        S = sim_scores(sentences)

    sentence_ranks = pagerank(S)
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(sentences)
    machine_summary = ""
    for each in summary:
        machine_summary = machine_summary+each+" "
    return machine_summary

<h2> Calculate Rouge </h2>

In [11]:
def rougeN(gold_summary,machine_summary,n):
    stemmer = SnowballStemmer("english")

    gold_summary_slammed = ""
    for word in gold_summary.split():
        gold_summary_slammed = gold_summary_slammed+stemmer.stem(word)+" "


    machine_summary_slammed = ""
    for word in machine_summary.split():
        machine_summary_slammed = machine_summary_slammed+stemmer.stem(word)+" "

    n_gold = ngrams(gold_summary_slammed.split(" "),n)
    n_machine = ngrams(machine_summary_slammed.split(" "),n)

    gold_list = []

    for gram in n_gold:
        gold_list.append(gram)

    machine_list = []
    for gram in n_machine:
        machine_list.append(gram)

    return float(len(list(set(gold_list).intersection(machine_list))))/float(len(list(set(gold_list))))

<h2> Find Summary </h2>

In [12]:
# def summary(SUMMARY_COMP_FACT,sentences):
#     scores = []
#     top_sentences = sorted(dict(sentences).items(), key=lambda x: x[1], reverse=True)[:(len(sentences)/SUMMARY_COMP_FACT)]
#     machine_summary = ""
#     for each in top_sentences:
#         machine_summary = machine_summary + each[0]
#     scores.append([SUMMARY_COMP_FACT, len(top_sentences), rougeN(gold_summary,machine_summary,2), rougeN(gold_summary,machine_summary,1)])

In [13]:
def summarize(sentences):
    paired_sens = {}
    num_sen = len(sentences)/6

    for pair in enumerate(sentences):
         paired_sens[pair[0]]=pair[1][1]

    temp = sorted((paired_sens).items(), key=lambda x: x[1], reverse=True)[:int(num_sen)]
    temp = sorted(dict(temp).items(), key=lambda x: x[0], reverse=False)[:int(num_sen)]
    machine_summary = ""
    for i in temp:
#         print sentences[i[0]][0],"\n"
        machine_summary = machine_summary +  sentences[i[0]][0] + "\n"
    return machine_summary

<h2> Main </h2>

In [ ]:
from nltk.corpus import brown, stopwords


df = pd.read_csv('news_summary.csv')
result = []
f = open("results.csv","w+")
f.write("article No. \t Luhn Rouge1 \t Luhn Rouge2 \t TextRank-Word_count Rouge1 \t TextRank-Word_count Rouge2 \t TextRank-word2vec Rouge1 \t TextRank-word2vec Rouge2")
for i in range(0,4300):
    print i
    f.write("\n")
    try:
        url = df['read_more'][i]
        gold_summary = df['text'][i]
        if "indiatoday" in url or "intoday" in url:
            baseurl = "i"
        elif "hindustantimes" in url:
            baseurl = "h"
        elif "theguardian" in url:
            baseurl = "t"

        file = urllib.urlopen(url)
        html = file.read()

        text = clean_data(html,baseurl)
        ws = word_scores(text)

        ss = sentence_score(text,ws)

        machine_summary_1 = summarize(ss)
        summary = []
        # TextRank with Word Count
        machine_summary_2 = textrank(text, "word_count",len(text)/5,stopwords=stopwords.words('english'))
        # TextRank with Word2Vec
        machine_summary_3 = textrank(text, "word2vec",len(text)/5,stopwords=stopwords.words('english'))
          
#         print "\n"
#         print i
#         print "Luhn's Approach    "+ "\t" +str(rougeN(gold_summary,machine_summary_1,2))+"\t"+str(rougeN(gold_summary,machine_summary_1,1))
#         print "TextRaNk - Word Count"+ "\t" + str(rougeN(gold_summary,machine_summary_2,2))+ "\t" +str(rougeN(gold_summary,machine_summary_2,1))
#         print "TextRank - Word2Vec"+ "\t" + str(rougeN(gold_summary,machine_summary_3,2))+ "\t" +str(rougeN(gold_summary,machine_summary_3,1))

        f.write(str(i)+"\t" +str(rougeN(gold_summary,machine_summary_1,1))+"\t"+str(rougeN(gold_summary,machine_summary_1,2)))
        f.write("\t" + str(rougeN(gold_summary,machine_summary_2,1))+ "\t" +str(rougeN(gold_summary,machine_summary_2,2)))
        f.write("\t" + str(rougeN(gold_summary,machine_summary_3,1))+ "\t" +str(rougeN(gold_summary,machine_summary_3,2)))

#         result.append([i, rougeN(gold_summary,machine_summary,2),rougeN(gold_summary,machine_summary,1)])
    except:
        print "UnicodeDecodeError"
        
f.close()

0


/home/shashwat/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if __name__ == '__main__':
/home/shashwat/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
UnicodeDecodeError
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
UnicodeDecodeError
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
UnicodeDecodeError
101
102
103
104
105
106
107
108
109
110
UnicodeDecodeError
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
UnicodeDecodeError
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
UnicodeDecodeError
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
UnicodeDecodeError
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244

/home/shashwat/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in divide
  from ipykernel import kernelapp as app
